In [1]:
import os
import random
import shutil
import time
import warnings
import zipfile

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!cp -r /content/drive/MyDrive/.kaggle /root/
!chmod 0600 /root/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d mengcius/cinic10

100% 752M/754M [00:23<00:00, 42.1MB/s]
100% 754M/754M [00:23<00:00, 33.7MB/s]


In [10]:
ZIPFILE = '/content/cinic10.zip'
ROOTDIR="/content/drive/MyDrive/cinic-10"
TRAINDIR="/content/drive/MyDrive/cinic-10/train"
VALDIR="/content/drive/MyDrive/cinic-10/valid"

In [ ]:
# Open the zip file and extract to ROOTDIR
if os.path.isfile(ZIPFILE) is True and os.path.exists(ROOTDIR) is False:
  # Create a directory using ROOTDIR
  os.makedirs(ROOTDIR, exist_ok=True)

  with zipfile.ZipFile(ZIPFILE, 'r') as zip_ref:
      # Extract the contents to the ROOTDIR directory
      zip_ref.extractall(ROOTDIR)

In [12]:
SEED=1

In [13]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [14]:
START_EPOCH = 0

### Set the architecture to resnet 18 below

In [21]:
##########################
ARCH = "resnet18"
# please look up how to do that
########################
EPOCHS = 5
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
PRINT_FREQ = 50
TRAIN_BATCH=128
VAL_BATCH=128
WORKERS=2

### Check if cuda is available here

In [16]:
# check if cuda is available in this cell
# if it is not available, you should not go forward!
if torch.cuda.is_available() is not True:
  print("No cuda device!")

### Assign your GPU below

In [17]:
# Assign your GPU in this cell
GPU = "cuda:0"

In [18]:
# set your active device to your GPU in this cell
device = 'cuda'


In [19]:
# enable algorithm optimization
cudnn.benchmark = True

### Fill in the heart of the train section below

In [22]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    ######################
    # switch model to train mode here
    model.train()
    ################

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        #####################
        # send the images to cuda device
        # send the target to cuda device
        images = images.to(device)
        target = target.to(device)
 
        # compute output
        # output = model ?? images
        output = model(images)

        # compute loss 
        # loss = criterion, output, target
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        
        #### zero out gradients in the optimier
        ## optimizer ..??
        optimizer.zero_grad()
        
        ## backprop!
        ### loss... ???
        loss.backward()
        
        # update the weights!
        ## optimier .. ??
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)

#### Fill in the validate section below

In [23]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            
            ### send the images and target to cuda
            images = images.to(device)
            target = target.to(device)

            # compute output
            # output = model ??? images?
            output = model(images)

            # compute loss
            # loss  = criterion ?? output ?? target
            loss = criterion(output, target)


            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

### Save the checkpoint

In [24]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    # save the model state!
    # state ??? 
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [25]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [26]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [27]:
# if we are adjusting the LR manually use this
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [28]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [29]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]

In [30]:
normalize = transforms.Normalize(mean=cifar_mean_RGB, std=cifar_std_RGB)

In [31]:
IMG_SIZE = 32
# IMG_SIZE = 224

### Initialize the model using the architecture you selected above

In [32]:
# select the model
# model = ... 
if ARCH == "resnet18":
    model = models.resnet18(pretrained=True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

### Send the model to the cuda device

In [33]:
# send the model to the cuda device.. 
model = model.to(device)

### Instantiate the loss to cross entropy

In [35]:
# use the cross-entropy loss
criterion = nn.CrossEntropyLoss()

### Instantiate the optimizer to SGD

In [36]:
# use SGD .. use the momentum and weight decay vars
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)


#### Create the learning rate scheduler

In [38]:
# use CosineAnnealingLR
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [39]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(cifar_mean_RGB, cifar_std_RGB),
])

### Create the train dataset object

In [43]:
# use CINIC-10
train_dataset = torchvision.datasets.ImageFolder(root=TRAINDIR, transform=transform_train)
#train_dataset = torchvision.datasets.CIFAR10('./datasets', train=True, 
#                                         download=True, transform=transform_train)



In [44]:
transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar_mean_RGB, cifar_std_RGB),
])

### Create the val dataset object

In [45]:
# use CINIC-10
val_dataset = torchvision.datasets.ImageFolder(root=VALDIR, transform=transform_val)

#val_dataset = torchvision.datasets.CIFAR10(root='./datasets', train=False,
#                                        download=True, transform=transform_val)

### Create the train dataloader

In [46]:
# fill this in
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH, 
                                           shuffle=True, num_workers=WORKERS)


### Create the c

In [47]:
# fill this in..
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=VAL_BATCH, 
                                          shuffle=False, num_workers=WORKERS)


In [48]:
best_acc1 = 0

In [ ]:
for epoch in range(START_EPOCH, EPOCHS):
    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))

Epoch: [0][  0/704]	Time 12.572 (12.572)	Data  3.170 ( 3.170)	Loss 1.3703e+01 (1.3703e+01)	Acc@1   0.00 (  0.00)	Acc@5   0.78 (  0.78)
Epoch: [0][ 50/704]	Time  0.276 ( 0.731)	Data  0.252 ( 0.515)	Loss 4.8737e+00 (4.8204e+00)	Acc@1  14.06 ( 10.71)	Acc@5  58.59 ( 50.74)
Epoch: [0][100/704]	Time  0.566 ( 0.777)	Data  0.536 ( 0.652)	Loss 2.2826e+00 (4.0181e+00)	Acc@1  14.84 ( 12.62)	Acc@5  68.75 ( 55.10)
